# Model Training
Train BGRU and GAN models for HK and US stock market, repectively

In [41]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional
from tensorflow.keras.layers import GRU
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

In [42]:
MODEL_STRUCTURE_PATH = "./diagrams/model_structures"
PROCESSED_STOCKS_PATH = "./data/processed"
EVALUATE_STOCKS_PATH = "./data/processed/stocks_for_evaluate"

def create_dir_if_not_exist(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname, exist_ok=True)

### Define models structure
##### BGRU models
###### Reference:
```
Salimath, S., Chatterjee, T., Mathai, T., Kamble, P., & Kolhekar, M. (2021, April). Prediction of Stock Price for Indian Stock Market: A Comparative Study Using LSTM and GRU. In International Conference on Advances in Computing and Data Sciences (pp. 292-302). Springer, Cham.
Lin, H., Chen, C., Huang, G., & Jafari, A. (2021). Stock price prediction using Generative Adversarial Networks. Journal of Computer Science, (17(3), 188–196. doi:10.3844/jcssp.2021.188.196
https://github.com/grudloff/stock_market_GAN
```

In [43]:
def make_bgru_model(input_dim, feature_cnt) -> tf.keras.models.Model:
    model = Sequential()

    # first gru + dropout layer
    model.add(
        Bidirectional(
            GRU(
                units=128,
                return_sequences=True,
                input_shape=(input_dim, feature_cnt)
                )
        )
    )
    model.add(
        Dropout(rate=0.3)
    )

    # second gru + dropout layer
    model.add(
        Bidirectional(
            GRU(
                units=32,
                return_sequences=True
                )
        )
    )
    model.add(
        Dropout(rate=0.3)
    )

    # output dense layer
    model.add(
        Dense(units = 1)
    )

    # compile model and use Adam optimizer
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="mse"
    )

    print(model.summary())
    return model

In [44]:
X_train_hk = np.load("{}/test_X_hk.npy".format(PROCESSED_STOCKS_PATH))
make_bgru_model(input_dim=X_train_hk[1], feature_cnt=[2])

TypeError: only integer scalar arrays can be converted to a scalar index